## Finding the Best Way to Swap Coins

In [1]:
import pandas as pd
from IPython.display import display, HTML

from src.data_extractors import get_pools, get_prices
from src.denom_utils import rename_denom
from src.swap_utils import generate_swap_bash_queries

source_coin = 'hydrogen'
target_coin = 'boot'
source_amount = 30_000_000_000
max_slippage = 0.15
min_path_decrease_premium = 0.005

source_coin = rename_denom(source_coin)
target_coin = rename_denom(target_coin)

#### Get Bostrom and Osmosis Pools Data

In [2]:
pools_df = get_pools(networks=['bostrom'], display_data=True)

Bostrom Pools


id,type_id,reserve_coin_denoms,reserve_account_address,pool_coin_denom,balances,swap_fee,network
1,1,"[boot, hydrogen]",bostrom1wrtkzr96362ty7ad0qrwhkpx743xcjrtv7j2cw,pool70D7610CBA8E94B27BAD7806EBD826F5626C486BBF5...,"[{'denom': 'boot', 'amount': '2356376420675'}, ...",0.003,bostrom
2,1,"[boot, milliampere]",bostrom1y7au5el59mf8mlak53gfvmr75gr2m2jz6lra0s,pool27BBCA67F42ED27DFFB6A450966C7EA206ADAA42BA0...,"[{'denom': 'boot', 'amount': '10386885645'}, {'...",0.003,bostrom
3,1,"[boot, tocyb]",bostrom1tkpsxklqu74eqsmezcwnc5hmfswrjgn9elcneq,pool5D83035BE0E7AB904379161D3C52FB4C1C392265AC1...,"[{'denom': 'boot', 'amount': '36944670050'}, {'...",0.003,bostrom
4,1,"[hydrogen, tocyb]",bostrom1kqkwg2eq9fc5r8f0n66fj6ekc5h3kkmqhthkqx,poolB02CE42B202A71419D2F9EB4996B36C52F1B5B60DAF...,"[{'denom': 'hydrogen', 'amount': '191501269101'...",0.003,bostrom
5,1,"[hydrogen, milliampere]",bostrom10xu73c3nkcdcf6j6u4j2kzq0cjftfj2nv7xz9y,pool79B9E8E233B61B84EA5AE564AB080FC492B4C953A9D...,"[{'denom': 'hydrogen', 'amount': '923050278388'...",0.003,bostrom
6,1,"[hydrogen, millivolt]",bostrom1u3u628ue3c5hn7ya4x4cjlk5t2d0unnarrq34k,poolE479A51F998E2979F89DA9AB897ED45A9AFE4E7DE32...,"[{'denom': 'hydrogen', 'amount': '536801646648'...",0.003,bostrom
7,1,"[hydrogen, uosmo in bostrom]",bostrom1np5jg5xq8x7axptrgad8dxd8spg8tavjh5vnaz,pool98692450C039BDD30563475A7699A7805075F592A36...,"[{'denom': 'hydrogen', 'amount': '1777283640214...",0.003,bostrom
8,1,"[hydrogen, deprecated uatom in bostrom]",bostrom1mgv368468aq8fsk2kynh0lpcsgtwa53zzlnhp0,poolDA191D1EBA3F4074C2CAB12777FC388216EED222F57...,[],0.003,bostrom
9,1,"[uosmo in bostrom, deprecated uatom in bostrom]",bostrom1tyx5u9mgwxzzj6rstsjxmfswpft75s0euh5ppu,pool590D4E176871842968705C246DA60E0A57EA41F9257...,[],0.003,bostrom
10,1,"[milliampere, millivolt]",bostrom1jphne3xq7335qvvepkupwcda8yyfp79ggfx7hy,pool906F3CC4C0F4634031990DB81761BD390890F8A8A80...,"[{'denom': 'milliampere', 'amount': '139518330'...",0.003,bostrom


#### Calculate Prices

In [3]:
price_df = get_prices(pools_df=pools_df, display_data=True)

,udsm in bostrom,milliampere,liquidpussy in bostrom,tocyb,pussy in bostrom,ujuno in bostrom,millivolt,uatom in bostrom,wei gravETH in bostrom,hydrogen,boot,deprecated uatom in bostrom,uosmo in bostrom,ugraviton in bostrom,aevmos in bostrom
udsm in bostrom,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.133754,NaN,NaN,NaN,NaN,NaN
milliampere,NaN,1,NaN,NaN,NaN,NaN,9.078076,NaN,NaN,0.000256,0.000714,NaN,NaN,NaN,NaN
liquidpussy in bostrom,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,4482.373271,NaN,NaN,NaN,NaN,NaN
tocyb,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,0.161535,0.45032,NaN,NaN,NaN,NaN
pussy in bostrom,NaN,NaN,NaN,NaN,1,NaN,NaN,5182780.089552,NaN,1595.532729,4446.353008,NaN,NaN,NaN,NaN
ujuno in bostrom,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.002848,NaN,NaN,NaN,NaN,NaN
millivolt,NaN,0.109496,NaN,NaN,NaN,NaN,1,NaN,NaN,0.000028,0.000078,NaN,NaN,NaN,NaN
uatom in bostrom,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1,0.0,0.000307,NaN,NaN,0.071855,NaN,NaN
wei gravETH in bostrom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8634088981.58478,1,NaN,NaN,NaN,NaN,NaN,NaN
hydrogen,7.431619,3884.53864,0.000222,6.153506,0.000623,349.078411,35382.497985,3237.902326,NaN,1,2.806342,NaN,232.821696,3.487629,0.0


#### Search the best way to swap coins

In [4]:
# main: coin_source -> coin_target
ways = []
pool_main_df = pools_df[(pools_df.reserve_coin_denoms.isin([[source_coin, target_coin]])) | (pools_df.reserve_coin_denoms.isin([[target_coin, source_coin]]))]
if len(pool_main_df) > 0:
    ways = [[[source_coin, target_coin]]]

# alternative: coin_source -> coin3 -> coin_target
coin3_list = list(price_df[(~price_df[source_coin].isna()) & (~price_df[target_coin].isna()) & (~price_df.index.isin([source_coin, target_coin]))].index)
for coin3 in coin3_list:
    ways.append([[source_coin, coin3], [coin3, target_coin]])

def way_to_str(way: list):
    return ", ".join(f"{way_item[0]} -> {way_item[1]}" for way_item in way)

print(f'ways count {len(ways)}')
print(''.join(f'{str(i + 1)}. {way_to_str(way)}\n' for i, way in enumerate(ways)))

ways count 5
1. hydrogen -> boot
2. hydrogen -> milliampere, milliampere -> boot
3. hydrogen -> tocyb, tocyb -> boot
4. hydrogen -> pussy in bostrom, pussy in bostrom -> boot
5. hydrogen -> millivolt, millivolt -> boot



In [5]:
def generate_all_swap_queries(amount_ways_list: list, print_message: bool = False) -> [float, list]:
    swap_amount = 0
    swap_queries = []
    for _way, _amount in amount_ways_list:
        _swap_amount_way, _swap_queries_way = generate_swap_bash_queries(_way, coin1_amount=_amount, pools_df=pools_df, price_df=price_df)
        swap_amount += _swap_amount_way
        swap_queries.append(_swap_queries_way)
        if print_message:
            print(f'{_way}\n{_swap_amount_way}\n')
    return swap_amount, swap_queries


def get_best_swap(ways: list,
                  amount = source_amount,
                  number_points: int = 10,
                  min_path_decrease_premium: float = min_path_decrease_premium,
                  print_message: bool = False) -> [pd.DataFrame, float]:
    if len(ways) == 0:
        return pd.DataFrame(columns=['way', 'amount', 'swap_queries']), 0.0
    elif len(ways) == 1:
        _amount_list = [amount]
    elif len(ways) == 2:
        _amount_list = [[coef_1 / number_points * amount, (number_points - coef_1) / number_points * amount]
                        for coef_1 in range(number_points + 1)]
    elif len(ways) == 3:
        _amount_list = [[coef_1 / number_points * amount, coef_2 / number_points * amount, (number_points - coef_1 - coef_2) / number_points * amount]
                       for coef_1 in range(number_points + 1)
                       for coef_2 in range(number_points - coef_1 + 1)]
    else:
        _amount_list = [[coef_1 / number_points * amount, coef_2 / number_points * amount, coef_3 / number_points * amount, (number_points - coef_1 - coef_2 - coef_3) / number_points * amount]
                       for coef_1 in range(number_points + 1)
                       for coef_2 in range(number_points - coef_1 + 1)
                       for coef_3 in range(number_points - coef_1 - coef_2 + 1)]

    _swap_amount_list = []
    for _amount_list_item in _amount_list:
        _amount_ways_list = [[_way, _amount] for _way, _amount in zip(ways, _amount_list_item)]
        _swap_result, _swap_queries = generate_all_swap_queries(_amount_ways_list)
        _swap_amount_list.append([_amount_list_item, _swap_result, _swap_queries])

    _swap_amount_df = pd.DataFrame(_swap_amount_list,
                                  columns = ['amount_by_way_item', 'swap_result', 'swap_queries'])\
                                  .sort_values('swap_result', ascending=False)
    _swap_amount_df['effective_swap_result'] = \
        _swap_amount_df.apply(
            lambda x: x['swap_result'] * (1 - min_path_decrease_premium * len([item for item in x['amount_by_way_item'] if item != 0])),
            axis=1)
    if print_message:
        display(HTML((_swap_amount_df.to_html(index=False, notebook=True, show_dimensions=False))))

    _swap_max_amount_list = list(_swap_amount_df.loc[_swap_amount_df['effective_swap_result'].idxmax()][['amount_by_way_item', 'swap_queries']])
    return pd.DataFrame(zip(ways, _swap_max_amount_list[0], _swap_max_amount_list[1]), columns=['way', 'amount', 'swap_queries']), \
        _swap_amount_df['swap_result'].max()

In [6]:
best_swap_df, max_result = get_best_swap(ways=ways, number_points=25)
display(HTML(best_swap_df.to_html(index=False, notebook=True, show_dimensions=False)))
print(f'Source amount {source_amount:>,} {source_coin}\nMax target amount {int(max_result):>,} {target_coin}\n')

for index, row in best_swap_df.iterrows():
    if row.amount > 0:
        print(f'{way_to_str(row.way)}')
        print(f'Amount {int(row.amount):>,} {source_coin}')
        print('\n'.join(row.swap_queries), '\n')

way,amount,swap_queries
"[[hydrogen, boot]]",3.000000e+10,[cyber tx liquidity swap 1 1 30000000000hydroge...
"[[hydrogen, milliampere], [milliampere, boot]]",0.000000e+00,[cyber tx liquidity swap 5 1 0hydrogen milliamp...
"[[hydrogen, tocyb], [tocyb, boot]]",0.000000e+00,[cyber tx liquidity swap 4 1 0hydrogen tocyb 7....
"[[hydrogen, pussy in bostrom], [pussy in bostro...",0.000000e+00,[cyber tx liquidity swap 15 1 0hydrogen ibc/43D...


Source amount 30,000,000,000 hydrogen
Max target amount 10,530,766,252 boot

hydrogen -> boot
Amount 30,000,000,000 hydrogen
cyber tx liquidity swap 1 1 30000000000hydrogen boot 3.227293272287 0.003 --from $WALLET --chain-id bostrom --gas 200000 --gas-prices 0.01boot --yes --node https://rpc.bostrom.cybernode.ai:443 --broadcast-mode block 

